# ネットワークを遺伝子単位で出力する


In [25]:
# Move up to top directory
import os
from pathlib import Path

print(os.getcwd())

while not Path("LICENSE").exists():
    os.chdir("../")

print(os.getcwd())

/mnt/e/Research/TSUMUGI
/mnt/e/Research/TSUMUGI


In [26]:
P = print
from pprint import pprint as PP
from collections import Counter as C
from pathlib import Path
from collections import defaultdict
from itertools import combinations
import csv
import json
import pickle
import networkx as nx
import shutil
import gzip

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
overlap = pickle.load(open("data/overlap/overlapped_ratios_filtered.pkl", "rb"))
df_overlap = pd.DataFrame(
    overlap, columns=["marker1", "marker2", "overlap_ratio", "overlapped_mp_number", "overlapped_mp"]
)
df_overlap  # 240358 rows × 5 columns

,marker1,marker2,overlap_ratio,overlapped_mp_number,overlapped_mp
0,0610010K14Rik,4933427D14Rik,0.500,3,[embryonic lethality prior to organogenesis (H...
1,0610010K14Rik,Acvr1,0.600,3,[embryonic lethality prior to organogenesis (H...
2,0610010K14Rik,Adss2,0.375,3,[embryonic lethality prior to organogenesis (H...
3,0610010K14Rik,Ahcy,0.500,3,[embryonic lethality prior to organogenesis (H...
4,0610010K14Rik,Anapc4,0.750,3,[embryonic lethality prior to organogenesis (H...
...,...,...,...,...,...
133276,Zfp503,Zwilch,0.208,5,"[abnormal embryo size (Homo), abnormal placent..."
133277,Zfp512,Zpld1,0.429,3,"[abnormal behavior (Homo), decreased thigmotax..."
133278,Zfp612,Zfp641,0.500,3,"[decreased mean corpuscular volume (Homo), hyp..."
133279,Zfp638,Znhit2,0.429,3,[embryonic lethality prior to organogenesis (H...


In [28]:
marker_mp = json.load(open("data/annotation/symbol_mptermname.json"))
marker_mp = pd.DataFrame(marker_mp.items(), columns=["marker_symbol", "mp_term_name"])
marker_mp  # 7746 rows × 2 columns

,marker_symbol,mp_term_name
0,0610010K14Rik,[embryonic lethality prior to organogenesis (H...
1,0610040J01Rik,"[abnormal heart morphology (Homo), abnormal sp..."
2,1110059G10Rik,[vertebral transformation (Homo)]
3,1500009L16Rik,"[decreased bone mineral density (Homo), increa..."
4,1600014C10Rik,"[abnormal coat/hair pigmentation (Homo), abnor..."
...,...,...
7741,Zwint,[embryonic lethality prior to organogenesis (H...
7742,Zyg11b,[decreased exploration in new environment (Het...
7743,Zzef1,"[abnormal coat/hair pigmentation (Homo), abnor..."
7744,Zzz3,"[abnormal cornea morphology (Hetero), cornea o..."


In [29]:
# DFをdictに変換
marker_mp_dict = dict(zip(marker_mp.marker_symbol, marker_mp.mp_term_name))

## ひとつの遺伝子をハンドルする


In [30]:
gene_symbol = "Rab10"

In [31]:
P(sum(df_overlap.marker1 == gene_symbol))
P(sum(df_overlap.marker2 == gene_symbol))

13
29


In [32]:
import networkx as nx

In [33]:
G = nx.from_pandas_edgelist(df_overlap, "marker1", "marker2")

In [34]:
# ノードAと直接つながっているノードのみを取得
neighbors = list(G.neighbors(gene_symbol))
subgraph_nodes = [gene_symbol] + neighbors
subgraph = G.subgraph(subgraph_nodes)

P(subgraph)

Graph with 43 nodes and 365 edges


In [35]:
P(subgraph.nodes())

['Lmbrd1', 'Ccdc59', 'Atp6v1b2', 'Ints8', 'Scfd1', 'Rab10', 'Haus4', 'Mcmbp', 'Srprb', 'Fbxo5', 'Dnm2', 'Atp2b1', 'Mccc2', 'Med25', 'Gtpbp4', 'Prpf39', 'Wee1', 'Atl2', 'Med10', 'Ints3', 'Rpain', 'Ebna1bp2', 'Cdca5', 'Clptm1', 'Thoc3', 'Cct3', 'Trappc11', 'Asxl1', 'Hars2', 'Rnaseh2a', 'Med28', 'U2af1', 'Tomm40', 'Chp1', 'Ppie', 'Sptb', 'U2af2', 'Ddx46', 'Btf3', 'Tomm22', 'Osbp', 'Dhx15', 'Tfrc']


In [36]:
# nodesを用意

node_json = []
for node in subgraph.nodes():
    annotation = marker_mp_dict[node]
    node_color = 1 if node == gene_symbol else 0
    node_json.append({"data": {"id": node, "label": node, "node_color": node_color, "annotation": annotation}})

P(node_json[0])

{'data': {'id': 'Lmbrd1', 'label': 'Lmbrd1', 'node_color': 0, 'annotation': ['abnormal embryo size (Hetero)', 'decreased mean corpuscular hemoglobin (Hetero)', 'embryonic lethality prior to organogenesis (Homo)', 'embryonic lethality prior to tooth bud stage (Homo)', 'preweaning lethality, complete penetrance (Homo)']}}


In [37]:
# edgesを用意
df_overlap["id"] = df_overlap.apply(lambda row: "-".join(sorted([row["marker1"], row["marker2"]])), axis=1)

edge_info = pd.DataFrame({"id": ["-".join(sorted([a, b])) for a, b in subgraph.edges()]}).merge(df_overlap, on="id")
edge_json = []
for edge in edge_info.itertuples():
    edge_json.append(
        {
            "data": {
                "source": edge.marker1,
                "target": edge.marker2,
                "edge_size": edge.overlap_ratio,
                "annotation": edge.overlapped_mp,
            }
        }
    )
P(edge_json[:3])

[{'data': {'source': 'Atp6v1b2', 'target': 'Lmbrd1', 'edge_size': 0.364, 'annotation': ['decreased mean corpuscular hemoglobin (Hetero)', 'embryonic lethality prior to organogenesis (Homo)', 'embryonic lethality prior to tooth bud stage (Homo)', 'preweaning lethality, complete penetrance (Homo)']}}, {'data': {'source': 'Btf3', 'target': 'Lmbrd1', 'edge_size': 0.231, 'annotation': ['decreased mean corpuscular hemoglobin (Hetero)', 'embryonic lethality prior to organogenesis (Homo)', 'preweaning lethality, complete penetrance (Homo)']}}, {'data': {'source': 'Cct3', 'target': 'Lmbrd1', 'edge_size': 0.375, 'annotation': ['decreased mean corpuscular hemoglobin (Hetero)', 'embryonic lethality prior to organogenesis (Homo)', 'preweaning lethality, complete penetrance (Homo)']}}]


In [38]:
# Cytoscape.js用のJSON形式に変換
cytoscape_data = node_json + edge_json

In [39]:
# JSONに変換
cytoscape_json = json.dumps(cytoscape_data, indent=2)
print(cytoscape_json)

[
  {
    "data": {
      "id": "Lmbrd1",
      "label": "Lmbrd1",
      "node_color": 0,
      "annotation": [
        "abnormal embryo size (Hetero)",
        "decreased mean corpuscular hemoglobin (Hetero)",
        "embryonic lethality prior to organogenesis (Homo)",
        "embryonic lethality prior to tooth bud stage (Homo)",
        "preweaning lethality, complete penetrance (Homo)"
      ]
    }
  },
  {
    "data": {
      "id": "Ccdc59",
      "label": "Ccdc59",
      "node_color": 0,
      "annotation": [
        "cataract (Hetero)",
        "decreased mean corpuscular volume (Hetero)",
        "embryonic lethality prior to organogenesis (Homo)",
        "preweaning lethality, complete penetrance (Homo)"
      ]
    }
  },
  {
    "data": {
      "id": "Atp6v1b2",
      "label": "Atp6v1b2",
      "node_color": 0,
      "annotation": [
        "decreased hemoglobin content (Hetero)",
        "decreased mean corpuscular hemoglobin (Hetero)",
        "decreased mean corpuscula

In [40]:
# Path("Rab10.json").write_text(cytoscape_json)

## 一括処理


In [41]:
output_dir = Path("data/network/gene_symbol")
# remove network directory
if output_dir.exists():
    shutil.rmtree(output_dir)

output_dir.mkdir(exist_ok=True, parents=True)


In [42]:
gene_symbols = df_overlap.marker1.unique().tolist()
P(gene_symbols)
gene_symbols += df_overlap.marker2.unique().tolist()
gene_symbols = list(set(gene_symbols))
gene_symbols.sort()  # 以下のfor文で、どこまで遺伝子が処理されたのか途中経過を見積もるためのソート
P(gene_symbols[:3])
P(len(gene_symbols)) # 4244

['0610010K14Rik', '0610040J01Rik', '1500009L16Rik', '1700003F12Rik', '1700008P02Rik', '1700010I14Rik', '1700025G04Rik', '1700030K09Rik', '1700034E13Rik', '1700069L16Rik', '1700093K21Rik', '2210408I21Rik', '2510002D24Rik', '2510009E07Rik', '2610008E11Rik', '2700054A10Rik', '2700097O09Rik', '2810408A11Rik', '2900026A02Rik', '3425401B19Rik', '4921509C19Rik', '4930444P10Rik', '4930447C04Rik', '4930591A17Rik', '4933402N03Rik', '4933427D14Rik', '4933434E20Rik', '6430548M08Rik', 'A1cf', 'A430005L14Rik', 'AA986860', 'AB124611', 'AU021092', 'AU040320', 'Aadac', 'Aadacl4', 'Aak1', 'Aars1', 'Aars2', 'Aasdhppt', 'Aass', 'Aatf', 'Abca1', 'Abca12', 'Abca16', 'Abca2', 'Abca5', 'Abcb11', 'Abcb4', 'Abcb9', 'Abcc2', 'Abcc5', 'Abcd3', 'Abce1', 'Abcg8', 'Abhd10', 'Abhd11', 'Abhd16a', 'Abhd17c', 'Abhd3', 'Abhd5', 'Abhd8', 'Abi2', 'Abi3bp', 'Abitram', 'Ablim1', 'Ablim3', 'Abo', 'Abtb1', 'Acad11', 'Acadsb', 'Acat2', 'Acbd3', 'Acbd5', 'Ace', 'Ace2', 'Ache', 'Ackr3', 'Acot3', 'Acox1', 'Acoxl', 'Acp2', 'Acsf2',

In [43]:
# 前処理
G = nx.from_pandas_edgelist(df_overlap, "marker1", "marker2")
df_overlap["id"] = df_overlap.apply(lambda row: "-".join(sorted([row["marker1"], row["marker2"]])), axis=1)

In [44]:
for gene_symbol in gene_symbols:
    # ノードAと直接つながっているノードのみを取得
    neighbors = list(G.neighbors(gene_symbol))
    subgraph_nodes = [gene_symbol] + neighbors
    subgraph = G.subgraph(subgraph_nodes)

    # nodesを用意
    node_json = []
    for node in subgraph.nodes():
        annotation = marker_mp_dict[node]
        node_color = 1 if node == gene_symbol else 0
        node_json.append({"data": {"id": node, "label": node, "node_color": node_color, "annotation": annotation}})

    # edgesを用意
    edge_info = pd.DataFrame({"id": ["-".join(sorted([a, b])) for a, b in subgraph.edges()]}).merge(df_overlap, on="id")
    edge_json = []
    for edge in edge_info.itertuples():
        edge_json.append(
            {
                "data": {
                    "source": edge.marker1,
                    "target": edge.marker2,
                    "edge_size": edge.overlap_ratio,
                    "annotation": edge.overlapped_mp,
                }
            }
        )

    ### EdgeとNodeを統合して、出力
    # Combine node and edge

    network_json = node_json + edge_json
    # Output as JSON fil
    if network_json:
        output_json = output_dir / f"{gene_symbol}.json.gz"
        with gzip.open(output_json, "wt", encoding="utf-8") as f:
            json.dump(network_json, f, indent=2)

# 6m 52s

In [45]:
%%bash

ls -lh data/network/gene_symbol/ | head # total 4.6G -> 133M

total 148M
-rwxrwxrwx 1 kuno kuno  87K Jan 14 15:18 0610010K14Rik.json.gz
-rwxrwxrwx 1 kuno kuno  50K Jan 14 15:18 0610040J01Rik.json.gz
-rwxrwxrwx 1 kuno kuno  944 Jan 14 15:18 1500009L16Rik.json.gz
-rwxrwxrwx 1 kuno kuno 7.9K Jan 14 15:18 1700003F12Rik.json.gz
-rwxrwxrwx 1 kuno kuno 1.2K Jan 14 15:18 1700008P02Rik.json.gz
-rwxrwxrwx 1 kuno kuno  13K Jan 14 15:18 1700010I14Rik.json.gz
-rwxrwxrwx 1 kuno kuno 116K Jan 14 15:18 1700025G04Rik.json.gz
-rwxrwxrwx 1 kuno kuno 2.5K Jan 14 15:18 1700030K09Rik.json.gz
-rwxrwxrwx 1 kuno kuno 329K Jan 14 15:18 1700034E13Rik.json.gz


In [46]:
Path("data/overlap/available_gene_symbols.txt").write_text("\n".join(gene_symbols) + "\n")
print(len(gene_symbols)) # 4416 -> 4244

4139


# Debug


In [ ]:
# path_overlap = Path("data", "overlap_ratios_filtered.csv")
# df_overlap = pd.read_csv(path_overlap)

overlap = pickle.load(open("data/overlapped_ratios_filtered.pkl", "rb"))
df_overlap = pd.DataFrame(
    overlap, columns=["marker1", "marker2", "overlap_ratio", "overlapped_mp_number", "overlapped_mp"]
)
P(len(df_overlap))  # 184250
df_overlap.head(3)

184250


,marker1,marker2,overlap_ratio,overlapped_mp_number,overlapped_mp
0,Rhd,Ap4e1,0.071,5,"[decreased circulating free fatty acids level,..."
1,Rhd,Asxl1,0.136,3,"[decreased hemoglobin content, decreased mean ..."
2,Rhd,Abcg8,0.091,4,"[decreased hemoglobin content, decreased mean ..."


In [ ]:
path_mp_terms = list(Path("data", "mp_term_name").glob("*.csv"))
print(path_mp_terms[:3])
print(len(path_mp_terms))  # 670

[PosixPath('data/mp_term_name/abnormal_abdominal_wall_morphology.csv'), PosixPath('data/mp_term_name/abnormal_adipose_tissue_amount.csv'), PosixPath('data/mp_term_name/abnormal_adrenal_gland_morphology.csv')]
670


In [ ]:
# path_mp_term = Path("data", "mp_term_name", "decreased_circulating_glucose_level.csv")
path_mp_term = Path("data", "mp_term_name", "increased_circulating_glucose_level.csv")
mp_term = path_mp_term.stem
print(mp_term)


increased_circulating_glucose_level


In [ ]:
df_marker_effect = pd.read_csv(path_mp_term)
df_marker_effect = df_marker_effect[["marker_symbol", "effect_size"]].drop_duplicates()

# effect_sizeがNanの行を削除
df_marker_effect = df_marker_effect.dropna(subset=["effect_size"])

print(len(df_marker_effect))

df_marker_effect.head(3)


73


,marker_symbol,effect_size
0,Zfp513,0.200596
1,Dbnl,0.828539
2,Oas1g,0.922229


In [ ]:
# print(df_mp["effect_size"].describe())

In [ ]:
# Absolute value of effect size

df_marker_effect.loc[:, "effect_size"] = df_marker_effect["effect_size"].abs()

In [ ]:
# print(df_overlap[df_overlap["intersect_count"] > 5 & df_overlap["overlap_ratio"] > 0.5])

In [ ]:
# df_overlap_filterd = df_overlap[df_overlap["overlapped_mp_number"] > 0]
# df_overlap_filterd = df_overlap[(df_overlap["overlap_ratio"] > 0.25)]
df_overlap_filterd = df_overlap

In [ ]:
df_filtered = df_overlap_filterd[
    (df_overlap_filterd["marker1"].isin(df_marker_effect["marker_symbol"]))
    & (df_overlap_filterd["marker2"].isin(df_marker_effect["marker_symbol"]))
]

In [ ]:
print(len(df_filtered))

df_filtered.head(3)

213


,marker1,marker2,overlap_ratio,overlapped_mp_number,overlapped_mp
2704,Zfyve19,Rnf10,0.079,5,"[abnormal behavior, decreased exploration in n..."
2712,Zfyve19,Usp31,0.136,3,"[decreased exploration in new environment, dec..."
2722,Zfyve19,Ky,0.273,3,"[abnormal behavior, decreased exploration in n..."


## 出力


### Node を作成する


In [ ]:
# merge mp term
df_marker1 = df_filtered[["marker1"]]
df_marker2 = df_filtered[["marker2"]]
df_node_marker1 = pd.merge(df_marker1, df_marker_effect, left_on="marker1", right_on="marker_symbol", how="inner")[
    ["marker_symbol"]
]
df_node_marker2 = pd.merge(df_marker2, df_marker_effect, left_on="marker2", right_on="marker_symbol", how="inner")[
    ["marker_symbol"]
]

df_node = pd.concat([df_node_marker1, df_node_marker2], axis=0).drop_duplicates()

import pickle

marker_mp = pickle.load(open("data/marker_mp.pkl", "rb"))
marker_mp = pd.DataFrame(marker_mp.items(), columns=["marker_symbol", "mp_term"])

df_node = pd.merge(df_node, marker_mp, how="inner", on="marker_symbol")
df_node = pd.merge(df_node, df_marker_effect, how="inner", on="marker_symbol")

df_node[:3]


,marker_symbol,mp_term,effect_size
0,Zfyve19,"[abnormal behavior, decreased exploration in n...",0.695784
1,Csnk1g2,"[abnormal auditory brainstem response, abnorma...",0.665518
2,Bbs5,"[abnormal bone structure, abnormal gait, abnor...",1.902677


In [ ]:
# JSON形式に変換
node_json = []
for _, row in df_node.iterrows():
    node_json.append(
        {
            "data": {
                "id": row["marker_symbol"],
                "label": row["marker_symbol"],
                "annotation": row["mp_term"],
                "node_color": row["effect_size"],
            }
        }
    )

print(node_json[:3])

[{'data': {'id': 'Zfyve19', 'label': 'Zfyve19', 'annotation': ['abnormal behavior', 'decreased exploration in new environment', 'decreased locomotor activity', 'decreased thigmotaxis', 'increased circulating glucose level'], 'node_color': 0.695783774710263}}, {'data': {'id': 'Csnk1g2', 'label': 'Csnk1g2', 'annotation': ['abnormal auditory brainstem response', 'abnormal lens morphology', 'abnormal retina blood vessel morphology', 'abnormal retina morphology', 'abnormal retina vasculature morphology', 'decreased total retina thickness', 'increased cardiac muscle contractility', 'increased circulating aspartate transaminase level', 'increased circulating glucose level', 'increased circulating phosphate level', 'increased circulating potassium level', 'increased heart rate variability', 'increased mean corpuscular volume', 'increased pulmonary respiratory rate', 'persistence of hyaloid vascular system'], 'node_color': 0.665518283420352}}, {'data': {'id': 'Bbs5', 'label': 'Bbs5', 'annotatio

### Edge を作成する


In [ ]:
df_edge = df_filtered[["marker1", "marker2", "overlap_ratio", "overlapped_mp"]]
print(len(df_edge))
df_edge.head(3)

213


,marker1,marker2,overlap_ratio,overlapped_mp
2704,Zfyve19,Rnf10,0.079,"[abnormal behavior, decreased exploration in n..."
2712,Zfyve19,Usp31,0.136,"[decreased exploration in new environment, dec..."
2722,Zfyve19,Ky,0.273,"[abnormal behavior, decreased exploration in n..."


In [ ]:
# JSON形式に変換
edge_json = []
for _, row in df_edge.iterrows():
    edge_json.append(
        {
            "data": {
                "source": row["marker1"],
                "target": row["marker2"],
                "annotation": row["overlapped_mp"],
                "edge_size": row["overlap_ratio"],
            }
        }
    )

print(edge_json[:3])

[{'data': {'source': 'Zfyve19', 'target': 'Rnf10', 'annotation': ['abnormal behavior', 'decreased exploration in new environment', 'decreased locomotor activity', 'decreased thigmotaxis', 'increased circulating glucose level'], 'edge_size': 0.079}}, {'data': {'source': 'Zfyve19', 'target': 'Usp31', 'annotation': ['decreased exploration in new environment', 'decreased locomotor activity', 'increased circulating glucose level'], 'edge_size': 0.136}}, {'data': {'source': 'Zfyve19', 'target': 'Ky', 'annotation': ['abnormal behavior', 'decreased exploration in new environment', 'increased circulating glucose level'], 'edge_size': 0.273}}]


### Edge と Node を統合して、出力


In [ ]:
# Combine node and edge

network_json = node_json + edge_json

In [ ]:
print(len(network_json))
print(network_json[:3])
print(network_json[-3:])

269
[{'data': {'id': 'Zfyve19', 'label': 'Zfyve19', 'annotation': ['abnormal behavior', 'decreased exploration in new environment', 'decreased locomotor activity', 'decreased thigmotaxis', 'increased circulating glucose level'], 'node_color': 0.695783774710263}}, {'data': {'id': 'Csnk1g2', 'label': 'Csnk1g2', 'annotation': ['abnormal auditory brainstem response', 'abnormal lens morphology', 'abnormal retina blood vessel morphology', 'abnormal retina morphology', 'abnormal retina vasculature morphology', 'decreased total retina thickness', 'increased cardiac muscle contractility', 'increased circulating aspartate transaminase level', 'increased circulating glucose level', 'increased circulating phosphate level', 'increased circulating potassium level', 'increased heart rate variability', 'increased mean corpuscular volume', 'increased pulmonary respiratory rate', 'persistence of hyaloid vascular system'], 'node_color': 0.665518283420352}}, {'data': {'id': 'Bbs5', 'label': 'Bbs5', 'annot

In [ ]:
# Output as JSON file
Path("data/network").mkdir(exist_ok=True, parents=True)
with open(f"data/network/{mp_term}.json", "w") as f:
    json.dump(network_json, f, indent=2)

In [32]:
marker_mp

,marker_symbol,mp_term
0,Dpf2,"[Abnormal craniofacial morphology, Abnormal em..."
1,Gna13,"[Abnormal embryo size, Abnormal visceral yolk ..."
2,Dynlrb2,"[Increased fasting circulating glucose level, ..."
3,Slc16a2,"[Decreased circulating cholesterol level, Decr..."
4,Galr1,"[Decreased lymphocyte cell number, Enlarged sp..."
...,...,...
7167,Adgrb1,[Increased circulating insulin level]
7168,Amigo3,[Enlarged heart]
7169,Hap1,"[Preweaning lethality, complete penetrance]"
7170,Slc23a3,[Abnormal lung morphology]


In [38]:
for data in node_json:
    if "Lepr" == data["data"]["id"]:
        print(data)

{'data': {'id': 'Lepr', 'label': 'Lepr', 'annotation': ['Abnormal bone structure', 'Abnormal gait', 'Abnormal locomotor behavior', 'Decreased bone mineral content', 'Decreased bone mineral density', 'Decreased circulating chloride level', 'Decreased exploration in new environment', 'Decreased heart weight', 'Decreased lean body mass', 'Decreased leukocyte cell number', 'Decreased locomotor activity', 'Decreased lymphocyte cell number', 'Decreased spleen weight', 'Enlarged liver', 'Female infertility', 'Impaired righting response', 'Increased anxiety-related response', 'Increased circulating alanine transaminase level', 'Increased circulating alkaline phosphatase level', 'Increased circulating aspartate transaminase level', 'Increased circulating calcium level', 'Increased circulating cholesterol level', 'Increased circulating fructosamine level', 'Increased circulating glucose level', 'Increased circulating hdl cholesterol level', 'Increased circulating iron level', 'Increased circulat

In [163]:
for data in edge_json:
    if "Herc1" == data["data"]["source"]:
        print(data)
    if "Herc1" == data["data"]["target"]:
        print(data)

{'data': {'source': 'Camsap3', 'target': 'Herc1', 'annotation': ['abnormal auditory brainstem response', 'abnormal lens morphology', 'cataract', 'decreased bone mineral content', 'decreased bone mineral density', 'decreased lean body mass', 'increased leukocyte cell number', 'increased total body fat amount', 'male infertility'], 'edge_size': 0.31}}
{'data': {'source': 'Prdm14', 'target': 'Herc1', 'annotation': ['decreased bone mineral content', 'decreased bone mineral density', 'decreased lean body mass', 'impaired glucose tolerance', 'increased total body fat amount', 'male infertility'], 'edge_size': 0.273}}
{'data': {'source': 'Kash5', 'target': 'Herc1', 'annotation': ['decreased bone mineral content', 'decreased bone mineral density', 'decreased lean body mass', 'increased leukocyte cell number', 'increased total body fat amount', 'male infertility'], 'edge_size': 0.286}}
{'data': {'source': 'Trim37', 'target': 'Herc1', 'annotation': ['abnormal auditory brainstem response', 'abnor

In [150]:
for data in edge_json:
    if "Stoml2" == data["data"]["source"]:
        print(data)
    if "Stoml2" == data["data"]["target"]:
        print(data)

{'data': {'source': 'Vgf', 'target': 'Stoml2', 'annotation': ['decreased prepulse inhibition', 'increased circulating alanine transaminase level', 'increased circulating aspartate transaminase level', 'male infertility'], 'edge_size': 0.25}}
{'data': {'source': 'Fbxo38', 'target': 'Stoml2', 'annotation': ['increased circulating alanine transaminase level', 'increased circulating aspartate transaminase level', 'male infertility'], 'edge_size': 0.214}}
{'data': {'source': 'Stoml2', 'target': 'Clcn1', 'annotation': ['decreased prepulse inhibition', 'increased circulating aspartate transaminase level', 'increased circulating bilirubin level', 'male infertility'], 'edge_size': 0.182}}
{'data': {'source': 'Stoml2', 'target': 'Lepr', 'annotation': ['increased circulating alanine transaminase level', 'increased circulating aspartate transaminase level', 'male infertility'], 'edge_size': 0.073}}
{'data': {'source': 'Stoml2', 'target': 'Atp13a3', 'annotation': ['increased circulating alanine tra